In [58]:
# Преобразуем данные в массив tensor
in_data = torch.tensor(np.array(X_train, dtype=np.float32))
targets = torch.tensor(np.array(y_train, dtype=np.float32))

in_data_test = torch.tensor(np.array(X_test, dtype=np.float32))
targets_test = torch.tensor(np.array(y_test, dtype=np.float32))

in_data.shape

torch.Size([1002, 167])

In [68]:
in_data

tensor([[-8.3610e-01, -7.1599e-01, -1.0835e+00,  ..., -1.8454e-01,
         -1.8454e-01, -6.0579e-01],
        [ 1.6015e+00,  5.7440e-01,  1.3750e+00,  ...,  5.4188e+00,
          5.4188e+00,  1.6507e+00],
        [ 5.4346e-01, -1.9096e-01,  9.6157e-01,  ..., -1.8454e-01,
         -1.8454e-01, -6.0579e-01],
        ...,
        [-1.4332e+00, -1.5604e+00,  3.0014e-01,  ..., -1.8454e-01,
         -1.8454e-01, -6.0579e-01],
        [-4.7872e-03, -3.4977e-01,  1.3750e+00,  ..., -1.8454e-01,
         -1.8454e-01, -6.0579e-01],
        [-2.2135e-01,  8.9221e-01,  9.6157e-01,  ..., -1.8454e-01,
         -1.8454e-01,  1.6507e+00]])

In [69]:
targets

tensor([ 3070.8086, 15161.5342,  8823.2793,  ...,  1964.7800,  7151.0918,
         5836.5205])

In [59]:
#Neural Network
class NET(nn.Module):
    def __init__(self):
        super(NET,self).__init__()
        self.fc1 = nn.Linear(167,100)
        self.fc2 = nn.Linear(100, 10)
        self.fc3 = nn.Linear(10,1)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

net = NET()
print(net)

NET(
  (fc1): Linear(in_features=167, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=1, bias=True)
)


In [50]:
#Neural Network
class NET(nn.Module):
    def __init__(self):
        super(NET,self).__init__()
        
        self.linaer_relu_stack = nn.Sequential(
                  
        nn.Linear(167,100),
        nn.Dropout(0.3),
        nn.BatchNorm1d(100),
        nn.ReLU(),
        nn.Linear(100,10),
        nn.Dropout(0.15),
        nn.BatchNorm1d(10),
        nn.ReLU(),
        nn.Linear(10,1)
        )
        
    def forward(self,x):
        x = self.linaer_relu_stack(x)
        return x
        

net = NET()
print(net)

NET(
  (linaer_relu_stack): Sequential(
    (0): Linear(in_features=167, out_features=100, bias=True)
    (1): Dropout(p=0.3, inplace=False)
    (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=10, bias=True)
    (5): Dropout(p=0.15, inplace=False)
    (6): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): Linear(in_features=10, out_features=1, bias=True)
  )
)


In [60]:
# Зададим функцию потерь (среднеквадратическая ошибка) и оптимизатор 
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(),lr=0.001)

In [61]:
in_data.shape, targets.shape, in_data_test.shape, targets_test.shape

(torch.Size([1002, 167]),
 torch.Size([1002]),
 torch.Size([335, 167]),
 torch.Size([335]))

In [62]:
def train(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i: i+batch_size]
        batch_y = targets[i: i+batch_size]
        
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
        
    print(loss)

In [63]:
train(in_data, targets, batch_size=16)

100%|█████████████████████████████████████████████████████████████████████████████████| 63/63 [00:00<00:00, 813.41it/s]

tensor(5.1166e+08, grad_fn=<MseLossBackward0>)


In [64]:
#Сохранение и вызов модели
model_p = 'model.pth'
torch.save(net, model_p)

net1 = torch.load(model_p)

In [65]:
y_pred = net(in_data_test)
y_pred = y_pred.detach().numpy()
r2_score(targets_test,y_pred)

-1.1122357904408475

In [66]:
def test(in_data_test, targets_test):
    
    with torch.no_grad():
        output = net(in_data_test).reshape(-1)

    return r2_score(targets_test, output)

In [67]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    print(test(in_data_test, targets_test))
print("Done!")

Epoch 1
-------------------------------
-1.1122357904408475
Epoch 2
-------------------------------
-1.1122357904408475
Epoch 3
-------------------------------
-1.1122357904408475
Epoch 4
-------------------------------
-1.1122357904408475
Epoch 5
-------------------------------
-1.1122357904408475
Done!
